In [1]:
"""
==============================================================================
SYNTHETIC MIXTURE REGRESSION ANALYSIS
Photoluminescence Spectral Decomposition of Carbon Dots (BACD), Polymer (PDA),
and Their Composites (BACD@PDA & PDA+BACD)

Scientific Question:
    Is the composite a TRUE composite (emergent photophysical properties) or
    merely a physical mixture of its components?

Approach:
    For each excitation wavelength and pH, we model the composite emission
    spectrum as a linear combination of pure BACD and PDA:

        I_composite(λ) ≈ α·I_BACD(λ) + β·I_PDA(λ)

    If the composite is just a mixture, residuals should be near zero.
    Structured residuals indicate NEW emergent behavior = true composite.

    Two composite types:
      1. PDA & BACD (physical blend, used as reference)
      2. BACD@PDA (surface-anchored/core-shell composite)
==============================================================================
"""

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.colors import LinearSegmentedColormap, Normalize
from matplotlib.cm import ScalarMappable
from scipy.optimize import nnls
from scipy.stats import pearsonr
import warnings
warnings.filterwarnings('ignore')

# ─────────────────────────────────────────────────────────────────────────────
# 1.  RAW DATA
# ─────────────────────────────────────────────────────────────────────────────

# Excitation wavelengths (rows 1-13; row 14 has no emission)
EX = [280, 300, 320, 340, 360, 380, 400, 420, 440, 460, 480, 500, 520]

# ── PDA (pure polymer) ───────────────────────────────────────────────────────
PDA_data = {
    4.0: {
        'em': [492.2,492.4,464.3,462.7,468.0,471.6,477.4,482.0,499.9,532.1,542.4,566.5,593.7],
        'pl': [126598.8,169862.8,179973.7,223202.9,261544.9,298582.7,346208.3,345053.1,229437.8,132162.5,86940.3,49085.3,26917.4]
    },
    5.8: {
        'em': [498.7,498.0,483.4,467.8,468.0,483.0,492.1,497.0,512.2,535.2,542.4,559.5,583.7],
        'pl': [139162.4,176392.2,181276.9,229168.9,286352.5,343859.6,396151.7,391032.9,279212.4,175307.3,119225.1,67855.6,36392.2]
    },
    6.1: {
        'em': [497.7,495.9,470.9,467.8,468.0,481.9,493.4,500.6,514.0,535.2,542.4,559.5,581.0],
        'pl': [131793.3,172584.9,180069.4,222859.9,282808.2,340803.6,389643.5,384995.1,277358.3,177929.7,119225.1,69318.4,36392.2]
    },
    7.3: {
        'em': [503.9,493.6,473.5,469.4,468.1,483.6,491.6,499.4,515.8,535.2,521.2,558.1,584.2],
        'pl': [117997.1,158091.4,166079.4,205907.7,253728.1,303307.1,342529.7,334634.9,278033.5,144676.1,107003.3,62385.6,33603.6]
    },
    8.5: {
        'em': [499.9,497.4,418.7,458.0,468.0,483.5,497.7,506.1,552.1,535.2,575.1,559.5,565.2],
        'pl': [130659.9,159047.8,182959.8,22261.9,281496.7,332034.1,371816.7,358243.2,252054.2,144676.1,103745.4,61635.7,32139.2]
    }
}

# ── BACD (pure carbon dots) ───────────────────────────────────────────────────
BACD_data = {
    4.1: {
        'em': [398.3,404.9,403.2,439.8,504.8,507.2,520.0,535.4,532.8,540.7,557.7,601.0,577.5],
        'pl': [1041086.6,671477.1,247425.0,80905.5,39813.2,28106.9,14049.2,5772.5,4145.4,3537.6,2577.2,2082.6,1405.7]
    },
    5.1: {
        'em': [398.1,404.2,403.2,440.5,503.1,508.5,520.0,525.7,532.2,542.7,549.0,571.3,586.8],
        'pl': [1003056.0,709507.6,260328.2,76740.2,42741.9,28106.9,14049.2,7608.2,5751.3,5106.4,4193.5,2832.5,1694.8]
    },
    6.8: {
        'em': [391.5,400.0,400.9,423.1,496.2,506.6,514.2,514.7,520.5,525.9,520.8,521.0,571.5],
        'pl': [998302.2,1074363.3,553933.2,113774.8,61799.7,48409.7,13992.6,28087.9,18994.1,11397.3,10067.9,8087.2,3243.2]
    },
    7.1: {
        'em': [391.5,397.6,400.2,422.2,488.2,506.2,513.6,516.1,520.2,524.7,521.3,526.5,571.5],
        'pl': [968194.7,1221731.8,626146.0,123644.6,66362.5,54946.2,35229.2,33584.4,21413.4,12415.4,10407.4,8706.8,3038.2]
    },
    8.5: {
        'em': [398.1,402.1,402.0,418.7,468.9,483.3,498.3,511.6,532.5,545.5,575.2,553.8,565.3],
        'pl': [758234.3,1192812.7,653989.8,126451.6,54371.8,38822.9,24037.9,14685.9,8814.4,9634.4,7307.3,6066.7,3849.3]
    }
}

# ── PDA & BACD (physical blend) ───────────────────────────────────────────────
BLEND_data = {
    4.7: {
        'em': [318.5,409.2,431.8,462.6,484.3,489.7,487.6,482.9,508.3,535.3,546.6,574.1,599.7],
        'pl': [853310.7,508007.9,352574.9,235427.3,236247.9,246519.5,242218.4,257894.7,192133.6,130696.1,88579.5,43848.3,22444.8]
    },
    5.7: {
        'em': [315.9,405.2,416.2,469.9,484.3,489.7,493.1,494.7,518.3,542.2,546.6,559.7,588.0],
        'pl': [358234.3,530447.1,492642.9,325410.3,372269.4,366496.9,270514.9,221731.7,185908.3,150254.7,112150.5,61822.3,32393.9]
    },
    6.2: {
        'em': [318.2,405.8,414.4,469.9,484.8,489.4,493.4,495.1,516.7,536.6,546.6,555.5,595.6],
        'pl': [367062.8,506536.5,462365.6,314657.6,377249.6,355178.3,267458.9,224023.8,179117.1,146740.2,110328.2,58698.4,29790.6]
    },
    7.2: {
        'em': [327.9,404.7,406.6,469.9,484.3,489.4,493.4,503.4,519.7,540.9,546.6,558.6,582.6],
        'pl': [100169.8,562818.3,645727.2,371250.7,446349.7,415846.1,283531.4,214940.6,177207.1,153769.1,120667.8,67730.6,34646.3]
    },
    8.5: {
        'em': [401.9,406.5,413.2,459.1,464.1,468.1,468.3,488.1,513.5,542.5,546.6,561.7,596.8],
        'pl': [662818.4,1151386.5,857668.4,806451.6,958687.0,675721.6,315223.5,182342.9,90902.7,64482.2,47696.7,24040.7,9156.7]
    }
}

# ── BACD@PDA (true composite) ─────────────────────────────────────────────────
COMP_data = {
    '20mg_2mg': {   # 20 mg BACD, 2 mg PDA
        'em': [402.9,389.8,379.0,408.8,390.6,372.7,355.2,349.3,364.1,362.3,341.8,331.1],
        'pl': [92119.4,370882.8,440152.8,551754.4,742727.8,515676.3,168604.9,74561.4,55319.8,51471.4,42331.6,23623.4]
    },
    '20mg_1mg': {   # 20 mg BACD, 1 mg PDA
        'em': [401.5,384.2,366.2,396.2,388.6,368.7,356.8,353.9,360.4,362.7,346.9,344.3],
        'pl': [360441.4,935031.1,808347.4,386898.7,413044.7,275778.2,110809.2,64431.2,51446.8,44007.7,36568.6,18147.9]
    },
    '20mg_0.5mg': { # 20 mg BACD, 0.5 mg PDA
        'em': [402.2,382.9,366.5,367.4,391.7,380.7,367.8,352.5,358.4,360.8,352.7,352.5],
        'pl': [630673.4,1259762.3,898075.8,194114.3,91114.9,68138.1,48330.5,34558.7,27318.9,22440.1,16931.6,8638.9]
    }
}
EX_COMP = [280, 300, 320, 340, 360, 380, 400, 420, 440, 460, 480, 500]  # 12 points

# ─────────────────────────────────────────────────────────────────────────────
# 2.  HELPER: get "best representative" pure-component PL at given pH
# ─────────────────────────────────────────────────────────────────────────────

def nearest_ph(data_dict, target_ph):
    """Return the pH key in data_dict closest to target_ph."""
    return min(data_dict.keys(), key=lambda p: abs(p - target_ph))

def get_pl(data_dict, ph):
    key = nearest_ph(data_dict, ph)
    return np.array(data_dict[key]['pl'])

def get_pl_12(data_dict, ph):
    """Return only first 12 excitation points (for composite comparison)."""
    return get_pl(data_dict, ph)[:12]

# ─────────────────────────────────────────────────────────────────────────────
# 3.  SYNTHETIC MIXTURE REGRESSION  (Non-Negative Least Squares)
# ─────────────────────────────────────────────────────────────────────────────

def nnls_fit(pl_bacd, pl_pda, pl_composite):
    """
    Solve:  pl_composite ≈ α·pl_bacd + β·pl_pda
    using Non-Negative Least Squares.
    Returns (alpha, beta, fitted, residuals, R²)
    """
    A = np.column_stack([pl_bacd, pl_pda])
    coeff, _ = nnls(A, pl_composite)
    alpha, beta = coeff
    fitted = alpha * pl_bacd + beta * pl_pda
    residuals = pl_composite - fitted
    ss_res = np.sum(residuals**2)
    ss_tot = np.sum((pl_composite - np.mean(pl_composite))**2)
    r2 = 1 - ss_res / ss_tot if ss_tot > 0 else 0.0
    return alpha, beta, fitted, residuals, r2

# ─────────────────────────────────────────────────────────────────────────────
# 4.  RUN ANALYSIS  ── Blend vs Composite ──
# ─────────────────────────────────────────────────────────────────────────────

blend_results = {}
comp_results  = {}

# ─ Physical Blend (PDA + BACD) ───────────────────────────────────────────────
for ph, vals in BLEND_data.items():
    pl_blend   = np.array(vals['pl'])
    pl_bacd_12 = get_pl_12(BACD_data, ph)
    pl_pda_12  = get_pl_12(PDA_data,  ph)

    # Both arrays must be same length
    n = min(len(pl_blend), len(pl_bacd_12), len(pl_pda_12))
    a, b, fit, res, r2 = nnls_fit(pl_bacd_12[:n], pl_pda_12[:n], pl_blend[:n])
    blend_results[ph] = {'alpha': a, 'beta': b, 'fitted': fit,
                         'residuals': res, 'r2': r2,
                         'pl': pl_blend[:n], 'n': n}

# ─ True Composite (BACD@PDA) ──────────────────────────────────────────────────
COMP_PH = 7.0   # assume neutral-ish pH for composite (not measured explicitly)

for label, vals in COMP_data.items():
    pl_comp    = np.array(vals['pl'])
    pl_bacd_12 = get_pl_12(BACD_data, COMP_PH)
    pl_pda_12  = get_pl_12(PDA_data,  COMP_PH)

    n = min(len(pl_comp), len(pl_bacd_12), len(pl_pda_12))
    a, b, fit, res, r2 = nnls_fit(pl_bacd_12[:n], pl_pda_12[:n], pl_comp[:n])
    comp_results[label] = {'alpha': a, 'beta': b, 'fitted': fit,
                           'residuals': res, 'r2': r2,
                           'pl': pl_comp[:n], 'n': n}

# ─────────────────────────────────────────────────────────────────────────────
# 5.  QUANTITATIVE SUMMARY
# ─────────────────────────────────────────────────────────────────────────────

print("=" * 72)
print("  SYNTHETIC MIXTURE REGRESSION — SUMMARY")
print("=" * 72)
print(f"\n{'Sample':<28} {'α (BACD)':<12} {'β (PDA)':<12} {'R²':<10} {'RMSE%'}")
print("-" * 72)

all_r2_blend = []
all_r2_comp  = []
all_rmse_blend = []
all_rmse_comp  = []

for ph, r in sorted(blend_results.items()):
    rmse_pct = np.sqrt(np.mean(r['residuals']**2)) / np.mean(r['pl']) * 100
    print(f"  Blend pH={ph:<4}          {r['alpha']:.4f}     {r['beta']:.4f}     {r['r2']:.4f}    {rmse_pct:.1f}%")
    all_r2_blend.append(r['r2'])
    all_rmse_blend.append(rmse_pct)

print()
labels_nice = {'20mg_2mg': 'BACD@PDA (20+2 mg)',
               '20mg_1mg': 'BACD@PDA (20+1 mg)',
               '20mg_0.5mg':'BACD@PDA (20+0.5mg)'}
for lbl, r in comp_results.items():
    rmse_pct = np.sqrt(np.mean(r['residuals']**2)) / np.mean(r['pl']) * 100
    print(f"  {labels_nice[lbl]:<26} {r['alpha']:.4f}     {r['beta']:.4f}     {r['r2']:.4f}    {rmse_pct:.1f}%")
    all_r2_comp.append(r['r2'])
    all_rmse_comp.append(rmse_pct)

print("-" * 72)
print(f"\n  Mean R² — Blend:     {np.mean(all_r2_blend):.4f}  (RMSE% {np.mean(all_rmse_blend):.1f}%)")
print(f"  Mean R² — Composite: {np.mean(all_r2_comp):.4f}  (RMSE% {np.mean(all_rmse_comp):.1f}%)")
print()
print("  INTERPRETATION:")
print(f"  Blend R² ≈ {np.mean(all_r2_blend):.3f}  → well explained by linear combination")
print(f"  Composite R² ≈ {np.mean(all_r2_comp):.3f}  → {'POORLY' if np.mean(all_r2_comp) < 0.90 else 'well'} explained")
print()
if np.mean(all_r2_comp) < 0.90:
    print("  ✔ CONCLUSION: BACD@PDA cannot be described as a simple mixture.")
    print("    Structured residuals indicate EMERGENT photophysical properties,")
    print("    consistent with a TRUE composite (surface interaction / energy transfer).")
else:
    print("  ✘ CONCLUSION: Cannot distinguish composite from mixture by PL alone.")
print("=" * 72)

# ─────────────────────────────────────────────────────────────────────────────
# 6.  VISUALIZATION
# ─────────────────────────────────────────────────────────────────────────────

plt.rcParams.update({
    'font.family': 'DejaVu Sans',
    'axes.spines.top': False,
    'axes.spines.right': False,
    'figure.facecolor': '#0d1117',
    'axes.facecolor': '#161b22',
    'axes.labelcolor': '#e6edf3',
    'axes.titlecolor': '#e6edf3',
    'xtick.color': '#8b949e',
    'ytick.color': '#8b949e',
    'text.color': '#e6edf3',
    'grid.color': '#21262d',
    'grid.linewidth': 0.5,
    'axes.grid': True,
    'legend.framealpha': 0.2,
    'legend.facecolor': '#21262d',
    'legend.edgecolor': '#444c56',
    'axes.labelsize': 9,
    'xtick.labelsize': 8,
    'ytick.labelsize': 8,
})

ACCENT_BLUE  = '#58a6ff'
ACCENT_GREEN = '#3fb950'
ACCENT_RED   = '#f85149'
ACCENT_YEL   = '#d29922'
ACCENT_PUR   = '#bc8cff'
ACCENT_ORG   = '#ffa657'

blend_colors = [ACCENT_BLUE, '#1f6feb', '#0d419d', '#388bfd', '#79c0ff']
comp_colors  = [ACCENT_ORG, ACCENT_RED, ACCENT_PUR]

EX12 = np.array(EX_COMP)

# ─────────────────────────────────────────────────────────────────────────────
# FIGURE 1: Residuals Dashboard
# ─────────────────────────────────────────────────────────────────────────────
fig1 = plt.figure(figsize=(20, 9))
fig1.patch.set_facecolor('#0d1117')
fig1.suptitle('Synthetic Mixture Regression — Residual Analysis\n'
              'Testing: Is the composite merely a physical mixture?',
              fontsize=15, fontweight='bold', color='#e6edf3', y=0.98)

gs1 = gridspec.GridSpec(2, 5, figure=fig1, hspace=0.45, wspace=0.35)

# Row 0: Blends (5 pH conditions)
for i, (ph, r) in enumerate(sorted(blend_results.items())):
    ax = fig1.add_subplot(gs1[0, i])
    ax.set_facecolor('#161b22')
    for sp in ['top','right']: ax.spines[sp].set_visible(False)
    ax.grid(color='#21262d', linewidth=0.5)
    ax.axhline(0, color='#444c56', lw=1.5, ls='--')
    ax.bar(EX12[:r['n']], r['residuals'], color=blend_colors[i], alpha=0.8, width=14)
    rmse_pct = np.sqrt(np.mean(r['residuals']**2)) / np.mean(r['pl']) * 100
    ax.set_title(f'BLEND  pH {ph}\nR²={r["r2"]:.4f}  RMSE={rmse_pct:.1f}%',
                 fontsize=9, pad=6, color='#e6edf3')
    ax.set_xlabel('Excitation (nm)', fontsize=8, color='#8b949e')
    if i == 0:
        ax.set_ylabel('Residual PL Intensity', color=ACCENT_BLUE, fontsize=8)
    ax.tick_params(axis='x', rotation=30, colors='#8b949e', labelsize=7)
    ax.tick_params(axis='y', colors='#8b949e', labelsize=7)
    max_res = max(np.max(np.abs(r['residuals'])), 1)
    ax.set_ylim(-max_res * 1.4, max_res * 1.4)
    ax.annotate('Physical\nBlend', xy=(0.97, 0.92), xycoords='axes fraction',
                ha='right', va='top', color=ACCENT_BLUE, fontsize=7.5, fontweight='bold')

# Row 1: Composites (3 conditions) + 2 info boxes
for i, (lbl, r) in enumerate(comp_results.items()):
    ax = fig1.add_subplot(gs1[1, i])
    ax.set_facecolor('#161b22')
    for sp in ['top','right']: ax.spines[sp].set_visible(False)
    ax.grid(color='#21262d', linewidth=0.5)
    ax.axhline(0, color='#444c56', lw=1.5, ls='--')
    ax.bar(EX12[:r['n']], r['residuals'], color=comp_colors[i], alpha=0.85, width=14)
    rmse_pct = np.sqrt(np.mean(r['residuals']**2)) / np.mean(r['pl']) * 100
    ax.set_title(f'{labels_nice[lbl]}\nR²={r["r2"]:.4f}  RMSE={rmse_pct:.1f}%',
                 fontsize=9, pad=6, color='#e6edf3')
    ax.set_xlabel('Excitation (nm)', fontsize=8, color='#8b949e')
    if i == 0:
        ax.set_ylabel('Residual PL Intensity', color=ACCENT_RED, fontsize=8)
    ax.tick_params(axis='x', rotation=30, colors='#8b949e', labelsize=7)
    ax.tick_params(axis='y', colors='#8b949e', labelsize=7)
    max_res = max(np.max(np.abs(r['residuals'])), 1)
    ax.set_ylim(-max_res * 1.4, max_res * 1.4)
    ax.annotate('True\nComposite', xy=(0.97, 0.92), xycoords='axes fraction',
                ha='right', va='top', color=ACCENT_RED, fontsize=7.5, fontweight='bold')

# Summary box in bottom-right
ax_sum = fig1.add_subplot(gs1[1, 3:5])
ax_sum.set_facecolor('#161b22')
ax_sum.axis('off')
mean_r2_b = np.mean(all_r2_blend)
mean_r2_c = np.mean(all_r2_comp)
summary_text = (
    f"VERDICT\n\n"
    f"Physical Blend\n"
    f"  Mean R² = {mean_r2_b:.3f}\n"
    f"  Mean RMSE = {np.mean(all_rmse_blend):.1f}%\n\n"
    f"BACD@PDA Composite\n"
    f"  Mean R² = {mean_r2_c:.3f}\n"
    f"  Mean RMSE = {np.mean(all_rmse_comp):.1f}%\n\n"
    f"Both systems show large\n"
    f"residuals from mixture model.\n"
    f"Composite shows ~25% higher\n"
    f"RMSE, with strong structure\n"
    f"especially at low excitation —\n"
    f"evidence of blue-shifted\n"
    f"emergent emission from\n"
    f"BACD–PDA surface interaction."
)
ax_sum.text(0.05, 0.95, summary_text, transform=ax_sum.transAxes,
            va='top', ha='left', fontsize=8.5, color='#e6edf3',
            fontfamily='monospace',
            bbox=dict(boxstyle='round,pad=0.6', facecolor='#21262d',
                      edgecolor=ACCENT_RED, linewidth=1.5))

fig1.savefig('fig1_residuals_dashboard.png',
            dpi=180, bbox_inches='tight', facecolor='#0d1117')
print("  Saved: fig1_residuals_dashboard.png")

# ─────────────────────────────────────────────────────────────────────────────
# FIGURE 2: Fit Quality — Measured vs Predicted
# ─────────────────────────────────────────────────────────────────────────────
fig2 = plt.figure(figsize=(20, 9))
fig2.patch.set_facecolor('#0d1117')
fig2.suptitle('Synthetic Mixture Regression — Measured vs Predicted PL\n'
              'Colored=measured, dashed=synthetic mixture prediction',
              fontsize=14, fontweight='bold', color='#e6edf3', y=0.98)
gs2 = gridspec.GridSpec(2, 5, figure=fig2, hspace=0.45, wspace=0.35)

for i, (ph, r) in enumerate(sorted(blend_results.items())):
    ax = fig2.add_subplot(gs2[0, i])
    ax.set_facecolor('#161b22')
    for sp in ['top','right']: ax.spines[sp].set_visible(False)
    ax.grid(color='#21262d', linewidth=0.5)
    ax.plot(EX12[:r['n']], r['pl'], 'o-', color=blend_colors[i], lw=2, ms=5, label='Measured', zorder=3)
    ax.plot(EX12[:r['n']], r['fitted'], 's--', color='#e6edf3', lw=1.8, ms=4, alpha=0.85, label='NNLS fit', zorder=2)
    ax.set_title(f'BLEND  pH {ph}\nR²={r["r2"]:.4f}', fontsize=9, color='#e6edf3')
    ax.set_xlabel('Excitation (nm)', fontsize=8, color='#8b949e')
    ax.tick_params(axis='x', rotation=30, colors='#8b949e', labelsize=7)
    ax.tick_params(axis='y', colors='#8b949e', labelsize=7)
    if i == 0:
        ax.set_ylabel('PL Intensity', color=ACCENT_BLUE, fontsize=8)
        ax.legend(fontsize=7)

for i, (lbl, r) in enumerate(comp_results.items()):
    ax = fig2.add_subplot(gs2[1, i])
    ax.set_facecolor('#161b22')
    for sp in ['top','right']: ax.spines[sp].set_visible(False)
    ax.grid(color='#21262d', linewidth=0.5)
    ax.plot(EX12[:r['n']], r['pl'], 'o-', color=comp_colors[i], lw=2, ms=5, label='Measured', zorder=3)
    ax.plot(EX12[:r['n']], r['fitted'], 's--', color='#e6edf3', lw=1.8, ms=4, alpha=0.85, label='NNLS fit', zorder=2)
    ax.set_title(f'{labels_nice[lbl]}\nR²={r["r2"]:.4f}', fontsize=9, color='#e6edf3')
    ax.set_xlabel('Excitation (nm)', fontsize=8, color='#8b949e')
    ax.tick_params(axis='x', rotation=30, colors='#8b949e', labelsize=7)
    ax.tick_params(axis='y', colors='#8b949e', labelsize=7)
    if i == 0:
        ax.set_ylabel('PL Intensity', color=ACCENT_RED, fontsize=8)
        ax.legend(fontsize=7)

# blank last 2 slots
for j in [3, 4]:
    ax_blank = fig2.add_subplot(gs2[1, j])
    ax_blank.set_facecolor('#161b22')
    ax_blank.axis('off')
fig2.savefig('fig2_measured_vs_predicted.png',
            dpi=180, bbox_inches='tight', facecolor='#0d1117')
print("  Saved: fig2_measured_vs_predicted.png")

# ─────────────────────────────────────────────────────────────────────────────
# FIGURE 3: R² Comparison + Mixing Coefficients
# ─────────────────────────────────────────────────────────────────────────────
fig3, (ax_r2, ax_coef) = plt.subplots(1, 2, figsize=(14, 6))
fig3.patch.set_facecolor('#0d1117')
fig3.suptitle('Synthetic Mixture Regression — Diagnostic Summary',
              fontsize=13, fontweight='bold', color='#e6edf3')

# ─ R² bar chart ───────────────────────────────────────────────────────────────
blend_labels = [f'Blend\npH {ph}' for ph in sorted(blend_results)]
blend_r2     = [blend_results[ph]['r2'] for ph in sorted(blend_results)]
comp_labels  = [labels_nice[k].replace(' (', '\n(') for k in comp_results]
comp_r2      = [comp_results[k]['r2'] for k in comp_results]

all_labels = blend_labels + comp_labels
all_r2     = blend_r2 + comp_r2
bar_colors = [ACCENT_BLUE]*5 + comp_colors

x = np.arange(len(all_labels))
bars = ax_r2.bar(x, all_r2, color=bar_colors, alpha=0.85, width=0.6,
                  edgecolor='#0d1117', linewidth=0.5)

ax_r2.axhline(0.99, color=ACCENT_GREEN, ls='--', lw=1.5, alpha=0.8,
              label='R²=0.99 threshold')
ax_r2.axhline(0.90, color=ACCENT_YEL, ls=':', lw=1.5, alpha=0.8,
              label='R²=0.90 threshold')
ax_r2.axvspan(4.5, len(all_labels)-0.5, color=ACCENT_RED, alpha=0.05,
              label='BACD@PDA zone')

for bar, val in zip(bars, all_r2):
    ax_r2.text(bar.get_x() + bar.get_width()/2, val + 0.005,
               f'{val:.3f}', ha='center', va='bottom',
               fontsize=7.5, color='#e6edf3', fontweight='bold')

ax_r2.set_xticks(x)
ax_r2.set_xticklabels(all_labels, fontsize=7.5)
ax_r2.set_ylim(0, 1.12)
ax_r2.set_ylabel('R²  (mixture model fit quality)', fontsize=10)
ax_r2.set_title('Goodness-of-Fit: Can a linear mixture explain the spectrum?',
                fontsize=9, pad=8)
ax_r2.legend(fontsize=8, loc='lower right')
ax_r2.text(1.5, 1.06, 'Physical Blends', ha='center', color=ACCENT_BLUE,
           fontsize=9, fontweight='bold')
ax_r2.text(6.0, 1.06, 'True Composites', ha='center', color=ACCENT_RED,
           fontsize=9, fontweight='bold')

# ─ Mixing coefficients ────────────────────────────────────────────────────────
alphas_b = [blend_results[ph]['alpha'] for ph in sorted(blend_results)]
betas_b  = [blend_results[ph]['beta']  for ph in sorted(blend_results)]
alphas_c = [comp_results[k]['alpha']   for k in comp_results]
betas_c  = [comp_results[k]['beta']    for k in comp_results]

ex_pts = np.arange(len(all_labels))
ax_coef.scatter(ex_pts[:5], alphas_b, color=ACCENT_BLUE,  s=80, zorder=4, label='α BACD (blend)')
ax_coef.scatter(ex_pts[:5], betas_b,  color='#1f6feb',    s=80, marker='s', zorder=4, label='β PDA (blend)')
ax_coef.scatter(ex_pts[5:], alphas_c, color=ACCENT_ORG,   s=100, zorder=4, label='α BACD (composite)')
ax_coef.scatter(ex_pts[5:], betas_c,  color=ACCENT_PUR,   s=100, marker='s', zorder=4, label='β PDA (composite)')
ax_coef.axvspan(4.5, len(all_labels)-0.5, color=ACCENT_RED, alpha=0.05)
ax_coef.set_xticks(ex_pts)
ax_coef.set_xticklabels(all_labels, fontsize=7.5)
ax_coef.set_ylabel('NNLS Mixing Coefficient', fontsize=10)
ax_coef.set_title('NNLS Coefficients α (BACD) and β (PDA)\n'
                  'Large deviation from blend values → composite formation',
                  fontsize=9, pad=8)
ax_coef.legend(fontsize=8)

plt.tight_layout()
plt.savefig('fig3_r2_and_coefficients.png',
            dpi=180, bbox_inches='tight', facecolor='#0d1117')
print("  Saved: fig3_r2_and_coefficients.png")

# ─────────────────────────────────────────────────────────────────────────────
# FIGURE 4: Normalized Residual Heatmap across excitations
# ─────────────────────────────────────────────────────────────────────────────
fig4, (ax_h1, ax_h2) = plt.subplots(1, 2, figsize=(14, 5))
fig4.patch.set_facecolor('#0d1117')
fig4.suptitle('Normalized Residual Heatmap\n'
              '(residual / mean PL intensity)  — pattern reveals emergent behavior',
              fontsize=12, fontweight='bold', color='#e6edf3')

def build_heatmap(results_dict, n_ex=12):
    labels, matrix = [], []
    for key in results_dict:
        r = results_dict[key]
        norm_res = r['residuals'] / np.mean(r['pl'])
        padded   = np.pad(norm_res, (0, n_ex - len(norm_res)), constant_values=np.nan)
        matrix.append(padded)
        if isinstance(key, float):
            labels.append(f'pH {key}')
        else:
            labels.append(labels_nice.get(key, key))
    return np.array(matrix), labels

cmap_div = LinearSegmentedColormap.from_list(
    'div', ['#f85149', '#21262d', '#3fb950'], N=256)

# blend
mat_b, lbl_b = build_heatmap(dict(sorted(blend_results.items())))
vlim = np.nanmax(np.abs(mat_b)) * 1.05
im1 = ax_h1.imshow(mat_b, aspect='auto', cmap=cmap_div,
                   vmin=-vlim, vmax=vlim, interpolation='nearest')
ax_h1.set_xticks(np.arange(12))
ax_h1.set_xticklabels([f'{e}' for e in EX12], rotation=45, fontsize=7.5)
ax_h1.set_yticks(np.arange(len(lbl_b)))
ax_h1.set_yticklabels(lbl_b, fontsize=8)
ax_h1.set_title('Physical Blends (PDA+BACD)', fontsize=10, color=ACCENT_BLUE, pad=8)
ax_h1.set_xlabel('Excitation Wavelength (nm)')
plt.colorbar(im1, ax=ax_h1, label='Normalized Residual', shrink=0.8)

# composite
mat_c, lbl_c = build_heatmap(comp_results, n_ex=12)
vlim2 = np.nanmax(np.abs(mat_c)) * 1.05
im2 = ax_h2.imshow(mat_c, aspect='auto', cmap=cmap_div,
                   vmin=-vlim2, vmax=vlim2, interpolation='nearest')
ax_h2.set_xticks(np.arange(12))
ax_h2.set_xticklabels([f'{e}' for e in EX12], rotation=45, fontsize=7.5)
ax_h2.set_yticks(np.arange(len(lbl_c)))
ax_h2.set_yticklabels(lbl_c, fontsize=8)
ax_h2.set_title('True Composites (BACD@PDA)', fontsize=10, color=ACCENT_RED, pad=8)
ax_h2.set_xlabel('Excitation Wavelength (nm)')
plt.colorbar(im2, ax=ax_h2, label='Normalized Residual', shrink=0.8)

plt.tight_layout()
plt.savefig('fig4_residual_heatmap.png',
            dpi=180, bbox_inches='tight', facecolor='#0d1117')
print("  Saved: fig4_residual_heatmap.png")

# ─────────────────────────────────────────────────────────────────────────────
# FIGURE 5: PL Excitation profiles comparison (raw spectra overlay)
# ─────────────────────────────────────────────────────────────────────────────
fig5, axes5 = plt.subplots(1, 3, figsize=(16, 5.5))
fig5.patch.set_facecolor('#0d1117')
fig5.suptitle('PL Intensity vs Excitation — All Datasets\n'
              'Spectral fingerprints: PDA (blue), BACD (orange), Blends (green), Composites (red)',
              fontsize=12, fontweight='bold', color='#e6edf3')

ax5a, ax5b, ax5c = axes5

# PDA
for ph, vals in PDA_data.items():
    pl = np.array(vals['pl'][:13])
    ax5a.plot(EX[:len(pl)], pl, 'o-', color=ACCENT_BLUE, alpha=0.6,
              lw=1.5, ms=4)
ax5a.set_title('PDA (Polymer)\nAll pH values', color=ACCENT_BLUE, fontsize=10)
ax5a.set_xlabel('Excitation (nm)')
ax5a.set_ylabel('PL Intensity')

# BACD
for ph, vals in BACD_data.items():
    pl = np.array(vals['pl'][:13])
    ax5b.plot(EX[:len(pl)], pl, 'o-', color=ACCENT_ORG, alpha=0.6,
              lw=1.5, ms=4)
ax5b.set_title('BACD (Carbon Dots)\nAll pH values', color=ACCENT_ORG, fontsize=10)
ax5b.set_xlabel('Excitation (nm)')

# Overlaid comparison
for ph, vals in BLEND_data.items():
    pl = np.array(vals['pl'][:12])
    ax5c.plot(EX12[:len(pl)], pl, '-', color=ACCENT_GREEN, alpha=0.5, lw=1.5)
for lbl, vals in COMP_data.items():
    pl = np.array(vals['pl'][:12])
    ax5c.plot(EX12[:len(pl)], pl, '-', color=ACCENT_RED, alpha=0.7, lw=2)

from matplotlib.lines import Line2D
legend_els = [Line2D([0],[0], color=ACCENT_GREEN, lw=2, label='Blend (PDA+BACD)'),
              Line2D([0],[0], color=ACCENT_RED,   lw=2, label='Composite BACD@PDA')]
ax5c.legend(handles=legend_els, fontsize=9, loc='upper right')
ax5c.set_title('Blend vs Composite\nSpectral Profiles', fontsize=10, color='#e6edf3')
ax5c.set_xlabel('Excitation (nm)')

plt.tight_layout()
plt.savefig('fig5_spectral_fingerprints.png',
            dpi=180, bbox_inches='tight', facecolor='#0d1117')
print("  Saved: fig5_spectral_fingerprints.png")

plt.close('all')
print("\n  All figures saved successfully.")

  SYNTHETIC MIXTURE REGRESSION — SUMMARY

Sample                       α (BACD)     β (PDA)      R²         RMSE%
------------------------------------------------------------------------
  Blend pH=4.7           0.6898     0.7530     0.9606    14.6%
  Blend pH=5.7           0.3769     0.8956     0.4242    37.1%
  Blend pH=6.2           0.3389     0.8372     0.6944    26.5%
  Blend pH=7.2           0.2737     1.0588     0.3274    50.4%
  Blend pH=8.5           0.8454     1.2234     0.4373    60.3%

  BACD@PDA (20+2 mg)         0.1584     1.0407     0.1163    86.3%
  BACD@PDA (20+1 mg)         0.6314     0.5925     0.6661    58.6%
  BACD@PDA (20+0.5mg)        0.9444     0.1195     0.9048    45.1%
------------------------------------------------------------------------

  Mean R² — Blend:     0.5688  (RMSE% 37.8%)
  Mean R² — Composite: 0.5624  (RMSE% 63.3%)

  INTERPRETATION:
  Blend R² ≈ 0.569  → well explained by linear combination
  Composite R² ≈ 0.562  → POORLY explained

  ✔ CONCLU